In [1]:
df = pd.read_csv("./data/total_data.csv")

In [2]:
df["장르"][df["장르"].isna()] = "['']" 
df["장르"] = df["장르"].apply(lambda x: eval(x))
df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

df["평점"] = df["평점"].astype("float")
df["평가수"] = df["평가수"].astype("int")

C = df["평점"].mean()
m = df["평가수"].quantile(0.6)
def weighted_vote_average(record):
    v = record["평가수"]
    R = record["평점"]
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)
df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

In [3]:
# 10개 영화 20개 -> 100개(한번에)
    # 10개 영화 20개 -> 20(10번) (상위 5개씩 )
# 방향성 1. 20개, 40개, 60개 중 가중치 평점 정렬 5개 ~ 10개 뽑기

# 방향성 2. 100개 중 겹치는 값들의 가중치 부여 혹은 추가.
    # 

# 방향성 3. 보다 많이 잡아서(100개?) 중 20개 랜덤으로 반환

# 배너를 나눈다.
    # 추천, 랜덤 유사한 영화...
    # 상위 10(추천), 10개(랜덤)

def recommendations3(titles, numbers = 20):
    # titles 예능인지 아닌지
    dic = {20:21, 40:42, 60:63, 80:84, 100:105}
    
    indexes = []
    for title in titles:
        indexes.append(df[df.제목 == title].index[0])
    print(indexes)
    similar_indexes = pd.read_csv(f"./data/특징_matrix_{numbers}.csv")
    similar_indexes = np.array(similar_indexes).reshape((-1, dic[numbers]))
    similar_indexes = similar_indexes[indexes,:]
    similar_indexes_ls = list(similar_indexes.reshape(-1))
    similar_indexes = set(similar_indexes_ls)  # 중복된 영화들을 제거/ 가중치 부여 # ["마블", "마블", "대부", "배트맨"]
    
    similar_indexes = np.array(list(similar_indexes))
    for i in indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거

    counter = {}
    for index in similar_indexes:
        counter[index] = similar_indexes_ls.count(index)

    ls = [[] for i in range(max(counter.values()))]
    for k, v in counter.items():
        ls[v - 1] += [k]

    for i, l in enumerate(ls):
        if i > 0:
            print(f"{i+1} : {l}")

    result = df.iloc[similar_indexes]
    for k, v in counter.items():
        if v > 1:
            print(k, v)
            result["가중치평점"][k] += v

    # return result.sort_values("가중치평점", ascending=False)[:20]
    return df.iloc[similar_indexes][:20]

In [117]:
# http://localhost:5000/api/contents/recommend
# 10개 영화 20개 -> 100개(한번에)
    # 10개 영화 20개 -> 20(10번) (상위 5개씩 )
# 방향성 1. 20개, 40개, 60개 중 가중치 평점 정렬 5개 ~ 10개 뽑기
# 방향성 2. 100개 중 겹치는 값들의 가중치 부여 혹은 추가.
# 방향성 3. 보다 많이 잡아서(100개?) 중 20개 랜덤으로 반환

# 배너를 나눈다.
    # 추천, 랜덤 유사한 영화...
    # 상위 10(추천), 10개(랜덤)
import random as rd

def recommendations4(titles):
    # titles 예능인지 아닌지
    
    indexes = []
    for title in titles:
        indexes.append(df[df.제목 == title].index[0])
    print(indexes)
    
    similar_indexes = pd.read_csv(f"./data/특징_matrix_60.csv")
    similar_indexes = np.array(similar_indexes)
    
    # 각 영화별 10개 추천
    first_recommends = [] 
    for index in indexes:
        first_recommends += list(similar_indexes[index,1:10])
    
    similar_indexes = similar_indexes[indexes,10:]
    similar_indexes_ls = list(similar_indexes.reshape(-1))
    similar_indexes = set(similar_indexes_ls)  # 중복된 영화들을 제거/ 가중치 부여 # ["마블", "마블", "대부", "배트맨"]
    
    similar_indexes = np.array(list(similar_indexes))
    for i in indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거
        
    for i in first_recommends:
        similar_indexes = similar_indexes[similar_indexes != i]  # 첫번쨰 추천된 영화 기준 제거

    counter = {}
    for index in similar_indexes:
        counter[index] = similar_indexes_ls.count(index)

    ## 출력용 코드(제거 필요)
    ls = [[] for i in range(max(counter.values()))]
    for k, v in counter.items():
        ls[v - 1] += [k]
    for i, l in enumerate(ls):
        if i > 0:
            print(f"{i+1} : {l}")
    ##
    result = df.iloc[similar_indexes]
    for k, v in counter.items():
        if v > 1:
            print(k, v)
            result["가중치평점"][k] += v*2

    second_recommends = list(result.sort_values("가중치평점", ascending=False)[:6].index)
    third_recommends = list(result.sort_values("가중치평점", ascending=False)[6:].index)
    
    total = []
    total += rd.sample(first_recommends, k=9)
    total += second_recommends
    total += rd.sample(third_recommends, k=5)
    print(total)

    return df.iloc[total]

In [63]:
df[df.제목 == "어느 날 밤에 생긴 일"].시놉시스[366]

'재산을 노리고 구혼을 한 비행사인 킹 웨슬리와 결혼하고자 하는 엘리를 아버지 알렉산더가 반대하고 요트에 가둬 두자 엘리는 헤엄을 쳐 해안으로 도망쳐 나와 뉴욕으로 가는 그레이 하운드 버스에 오른다. 그 버스에는 근무 중에 술을 마셔 해고당한 신문 기자 피터가 맨 뒷 좌석에 자리잡고 있는데, 엘리는 피터와 맨 뒤의 좌석에 나란히 앉게 된다. 여행 도중 엘리가 소지품이 든 가방을 잃어버렸 음에도 신고하기를 거부하자 피터는 엘리의 신분에 대해 의심을 하게 된다. 그레 이하운드로 계속되는 여행 다음날 둘은 잠시 쉰 휴게실에서 느긋하게 아침 식사를 하다가 엘리와 피터는 버스를 놓치게 되고, 그런 와중 피터는 엘리의 신분을 알게 된다. 엘리는 피터에게 뉴욕에 도착하도록 도와주면 자신에 대한 특종 기사를 쓰게 해 주겠다며 거래를 제안하지만 부잣집 딸이나 부를 탐내 결혼하려는 킹 웨슬리 같은 사람한테 전혀 관심이 없다면서 퉁명스럽게 이 제안을 거절한다. 하지만 북쪽 으로 향하는 여행에서 몇 가지 우연찮은 사고가 일어나게 되고 피터와 엘리는 여러 곤경에 같이 빠지게 된다. 그러면서 둘은 서로 사랑에 빠지게 된다.'

In [72]:
df[df.제목 == "아이언맨"].시놉시스[3332]

'천재적인 두뇌와 재능으로 세계 최강의 무기업체를 이끄는 CEO이자, 타고난 매력으로 셀러브리티 못지않은 화려한 삶을 살아가던 억만장자 토니 스타크. 아프가니스탄에서 자신이 개발한 신무기 발표를 성공리에 마치고 돌아가던 그는 게릴라군의 갑작스런 공격에 의해 가슴에 치명적인 부상을 입고 게릴라군에게 납치된다. 가까스로 목숨을 건진 그에게 게릴라군은 자신들을 위한 강력한 무기를 개발하라며 그를 위협한다. 그러나 그는 게릴라군을 위한 무기 대신, 탈출을 위한 무기가 장착된 철갑수트를 몰래 만드는 데 성공하고, 그의 첫 수트인 Mark 1를 입고 탈출에 성공한다.'

In [64]:
df[df.제목 == "로보캅"].시놉시스[6300]

'범죄와 무질서로 혼란에 빠진 도시. 좋은 아빠이자 실력 있는 경찰로 평범하게 살아가던 ‘알렉스 머피’는 예기치 못한 사고로 인해 온 몸에 치명적 부상을 입는다. 로봇 테크놀로지의 기술을 가진 다국적 기업 ‘옴니코프’ 사(社)는 아내 ‘클라라’에게 ‘머피’의 몸에 최첨단 하이테크 수트를 장착할 것을 제안하고, 그녀는 목숨을 잃을 위기에 놓인 남편을 살리기 위해 제안을 받아들인다. 모두가 기다려온 완벽한 히어로 ‘로보캅’으로 재탄생한 ‘머피’는 ‘옴니코프’사의 체계적 훈련을 받으며 더욱 강력해지고, ‘클라라’는 기계처럼 변해버린 남편의 모습에 혼란스러워 한다. 한 치의 오차 없는 수트의 통제를 받으며 명령을 따라야 하는 ‘로보캅’. 하지만 그는 점차 스스로 수트를 지배하기 시작하고 그와 동시에 자신을 새롭게 태어나게 한 수트와 도시 이면에 거대한 음모가 숨겨져 있음을 깨닫게 되는데...!'

In [118]:
recommendations4(["아이언맨"])[["제목", "장르", "시놉시스", "가중치평점"]]

[3332]
[9739, 14989, 6300, 665, 11984, 1585, 4900, 10289, 6282, 3353, 8717, 7893, 6679, 1895, 3461, 8417, 8027, 9454, 3131, 1183]


,제목,장르,시놉시스,가중치평점
9739,스파이더맨: 홈커밍,SF 액션,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...,7.388819
14989,23,전쟁 드라마,23명의 이란 소년병들은 목숨걸고 참전했다가 포로로 붙잡혀 이라크 감옥에 가게 된다...,5.840253
6300,로보캅,액션 범죄 SF 스릴러,범죄와 무질서로 혼란에 빠진 도시. 좋은 아빠이자 실력 있는 경찰로 평범하게 살아가...,6.095403
665,슈퍼맨 4 - 최강의 적,액션 SF 가족,"천재적인 악당 렉스 루터는 수퍼맨과 대항할 수 있는 '핵인간'을 창조해낸다. 한편,...",3.871220
11984,언노운 어택,공포 스릴러 전쟁 액션 드라마 Made in Europe,"대서양 한복판, 깊은 바다속, 스웨덴 잠수함 '바다사자호'는 생화학 무기를 극비리에...",5.840253
1585,무장원 소걸아,액션 코미디 드라마,"무기력한 거지 부자, 왕의 역적을 무공으로 처치해라! 광주의 장군을 아버지로 둔 부...",6.294430
4900,아이언맨 2,SF 액션 Made in Europe,"세계 최강의 무기업체를 이끄는 CEO이자, 타고난 매력으로 화려한 삶을 살아가던 토...",6.994006
10289,스트라튼,스릴러 액션,영국해군특수부대 MI6 소속 부서인 스페셜 보트 대대 SBS의 요원 ‘존 스트라튼’...,5.320126
6282,아이언맨 3,SF 액션,"어벤저스의 일원으로 활약한 뉴욕 사건 이후, 트라우마로 인해 영웅으로서의 삶에 회의...",7.093756
3353,좀비랜드,코미디 액션 공포 판타지 SF,유난히 겁 많고 언제 어디서나 두려움과 불안 속에서 살아가고 있는 청년이 있다. 외...,7.587248


In [60]:
recommendations4(["스파이더맨", "신세계", "베테랑", "귀멸의 칼날", "겨울왕국"])[["제목", "장르", "시놉시스", "가중치평점"]]

[1657, 6255, 7787, 17205, 6699]
[6903, 11039, 11039, 7790, 2431, 476, 9978, 5606, 11974, 16050, 7771, 3397, 17256, 9739, 11042, 7978, 4349, 15771, 17046, 7978]


,제목,장르,시놉시스,가중치평점
6903,추억의 마니,드라마 애니메이션 가족,12살 소녀 안나는 요양차 방문한 바닷가 마을에서 어디서 본 듯한 낡은 저택을 발견...,7.518561
11039,스파이더맨: 뉴 유니버스,애니메이션 SF 액션 코미디 가족 판타지,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...,8.377196
11039,스파이더맨: 뉴 유니버스,애니메이션 SF 액션 코미디 가족 판타지,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...,8.377196
7790,오피스,스릴러 공포,어느 날 한 가족의 가장이자 착실한 회사원인 김병국 과장이 일가족을 살해하고 사라졌...,5.893502
2431,브링 다운 더 하우스,코미디,부인과 이혼하고 일에 치여 사는 인생에 아무런 재미가 없는 이혼 법률 전문 변호사 ...,5.624025
476,007 뷰 투 어 킬,스릴러 음악 액션,007은 시베리아 눈 속에 파묻힌 003의 시체에서 마이크로칩을 발견한다. 핵폭탄이...,6.281235
9978,올라프의 겨울왕국 어드벤처,코미디 판타지,올라프가 특별한 크리스마스 가족 전통이 없는 엘사와 안나를 위해 아렌델 왕국의 크리...,5.840253
5606,노바디 웍스,드라마,음향 전문가인 '피터'와 정신과 의사인 '줄리'는 아이들과 함께 LA에서 풍족하고도...,5.570126
11974,트루와 무지개 왕국의 겨울 축제,가족 애니메이션 액션,무지개 왕국에 얼음 수정 아이시가 찾아왔어요. 주위의 모든 것을 꽁꽁 얼릴 수 있는...,5.840253
16050,진격의 거인,SF 액션 드라마 판타지 공포 애니메이션,100여 년 전 갑자기 나타난 거인들에 의해 멸종 위기에 처한 인류는 거대한 벽을 ...,8.950435


In [61]:
# 유사도 높아도, 평점이 낮으면 ...

In [62]:
recommendations3(["스파이더맨"])[["제목", "장르", "시놉시스", "가중치평점"]]

[1657]


,제목,장르,시놉시스,가중치평점
11779,고공 침투: 노 리턴,액션 드라마 스릴러 Made in Europe,항공기 부품을 설치하는 기업의 기술자인 피터는 어느 날 상사 제이슨으로부터 부품 교...,6.520126
9739,스파이더맨: 홈커밍,SF 액션,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...,7.388819
14229,싸이코패스 스토커 살인,공포,미국 서부 사막의 평화로운 시골 마을 브라운스빌에서 난데없이 한 여성이 끔찍하게 살...,5.840253
8856,피터와 드래곤,판타지 코미디 드라마 가족 액션,갑작스런 사고로 부모를 잃고 비밀로 가득한 숲 속에서 혼자 살아가게 된 꼬마 ‘피터...,6.642684
7839,팬,판타지 코미디 가족,"피터팬, 그 전설이 시작된다! “이야기를 하나 해줄게요. 어른이 되지 않는 아이, ...",5.708500
11039,스파이더맨: 뉴 유니버스,애니메이션 SF 액션 코미디 가족 판타지,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...,8.377196
11042,스파이더맨: 파 프롬 홈,SF 액션 코미디,"어벤져스 멤버들과 타노스와의 대접전을 마친 뒤, 지난 5년 동안 사라졌던 사람들이 ...",7.384557
14772,타임스톤: 시간여행자,스릴러 드라마,"1970년, 미국에서 발사한 미사일이 멕시코의 ‘침묵 지대’에 추락한다. 미사일 잔...",5.840253
6465,메가 스파이더,SF 스릴러 액션 코미디,美 군사 연구소에서 비밀리에 진행되던 외계인과 거미의 유전자 변형 실험 중 돌연변이...,5.496456
6856,윈터스 테일,판타지 로맨스 드라마,"1895년 뉴욕에서는 폐병이 유행하였고, 피터의 아버지는 병으로 입국이 금지되자 갓...",6.174755


In [211]:
df[["제목", "장르", "시놉시스", "가중치평점"]]

,제목,장르,시놉시스,가중치평점
0,나 홀로 집에,코미디 가족,크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...,7.586332
1,크리스마스 대소동,코미디 Made in Europe,"클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...",7.560676
2,고스트 버스터즈,판타지 SF 액션 코미디,초심리를 연구하는 뉴욕의 괴짜교수 피터 밴크맨(빌 머레이)와 레이몬드 스탠드(댄 애...,7.779207
3,고스트 버스터즈 2,판타지 액션 코미디 SF,고스트 버스터즈 4인방은 악마 '고저'로부터 세계를 구한 후 순탄한 생활을 하지 못...,6.584805
4,멋진 인생,판타지 드라마 가족,조지 베일리는 베드폴드 마을 사람들을 위해 평생을 봉사하며 살아온 인물. 그는 항상...,8.574506
...,...,...,...,...
19139,대주재,SF,악을 징벌하는 대영웅으로 자라나는 특출한 소년 목진의 성장기,5.840253
19140,카란 조하르의 사랑이 뭔데!,RealityTV,"영화에서는 잘만 풀리는 로맨스, 현실에선 왜 이리 어려울까? 도대체 ‘사랑이 뭔데!...",5.840253
19141,M 사랑해야 할 사람이 있어서,RealityTV 음악,일본의 디바 탄생에 숨겨진 사랑과 이별 이야기,5.840253
19142,배틀피시,RealityTV,"여름, 미국 태평양 연안 북서부의 바다가 금광으로 변신한다! 값비싼 참치 떼를 잡기...",5.840253


In [119]:
# 평가 : 시리즈물, 장르가, 출연 배우, 감독, 스토리

list(df.iloc[[10549, 5941]]["제목"])

범죄_스릴러 = ["타짜", "신세계", "검사외전", "베테랑"]
가족_코미디 = ["몬스터 주식회사", "겨울왕국", "슈렉", "인사이드 아웃"]
범죄_스릴러_가족_코미디 = 범죄_스릴러 + 가족_코미디

In [120]:
recommendations4(범죄_스릴러_가족_코미디)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787, 2002, 6699, 1783, 7755]
2 : [8293, 4349, 10549, 6492, 8763, 4744, 5135, 3314, 7450, 3463, 7799, 16013, 9888, 18147, 16144, 8101]
3 : [11048]
8293 2
4349 2
10549 2
6492 2
8763 2
4744 2
11048 3
5135 2
3314 2
7450 2
3463 2
7799 2
16013 2
9888 2
18147 2
16144 2
8101 2
[13612, 7637, 15155, 12004, 7855, 14591, 476, 14434, 2162, 11048, 3314, 8101, 7799, 9888, 8763, 3393, 2185, 11142, 16852, 7902]


,제목,장르,가중치평점
13612,블랙머니,범죄 드라마 스릴러,5.840253
7637,더 매직: 리틀톰과 도둑공주,판타지 Made in Europe 애니메이션,5.840253
15155,인질,스릴러,5.840253
12004,더 트로프,스릴러 범죄 드라마 액션,5.840253
7855,몬스터 호텔 2,판타지 코미디 공포 드라마 가족 로맨스 음악 애니메이션 액션,6.671579
14591,휴가,드라마,5.840253
476,007 뷰 투 어 킬,스릴러 음악 액션,6.281235
14434,타짜: 원 아이드 잭,드라마,5.840253
2162,태양은 없다,드라마,5.840253
11048,다만 악에서 구하소서,액션 범죄 스릴러,6.251573


In [230]:
recommendations3(범죄_스릴러, 40)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787]
2 : [5217, 4770, 10419, 5816, 14556, 4349, 11048, 10549, 5941, 6016, 3463]
5217 2
4770 2
10419 2
5816 2
14556 2
4349 2
11048 2
10549 2
5941 2
6016 2
3463 2


,제목,장르,가중치평점
2048,아이스 스톰,드라마,7.294254
13314,여타짜,범죄,5.840253
9733,굿바이 싱글,코미디 드라마,5.840253
5128,레드 라이트,스릴러 액션 공포 로맨스 드라마 판타지 Made in Europe,6.177516
16405,살인자 만들기,범죄 다큐멘터리,8.482564
4118,혈의 누,스릴러,5.992202
11800,언더독,드라마 액션 스릴러 범죄 애니메이션,5.840253
17432,Eojjeoda Balgyeonhan Haru,코미디 판타지 로맨스 액션 SF,6.493502
4632,공공의 적 2,코미디 액션 드라마,5.840253
16936,유닛 42,범죄 스릴러 드라마 Made in Europe,6.226835


In [195]:
중복 = [11048, 15155]
list(df.iloc[중복]["제목"])

['다만 악에서 구하소서', '인질']

In [231]:
recommendations3(범죄_스릴러, 100)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787]
2 : [16450, 4209, 8358, 10419, 14556, 4349, 10549, 6492, 8648, 8763, 4677, 6725, 4744, 13193, 5005, 7116, 7279, 3206, 3397, 13693, 3463, 3556, 5738, 7790, 7799, 9888, 5816, 7872, 18147, 16144, 5941, 6016, 6052, 16301]
3 : [4118, 4770, 11048, 15155, 5217]
4118 3
16450 2
4209 2
8358 2
10419 2
14556 2
4349 2
10549 2
6492 2
8648 2
8763 2
4677 2
6725 2
4744 2
4770 3
11048 3
15155 3
13193 2
5005 2
7116 2
5217 3
7279 2
3206 2
3397 2
13693 2
3463 2
3556 2
5738 2
7790 2
7799 2
9888 2
5816 2
7872 2
18147 2
16144 2
5941 2
6016 2
6052 2
16301 2


,제목,장르,가중치평점
2048,아이스 스톰,드라마,7.294254
3,고스트 버스터즈 2,판타지 액션 코미디 SF,6.584805
16405,살인자 만들기,범죄 다큐멘터리,8.482564
4118,혈의 누,스릴러,5.992202
2081,나크,스릴러 코미디 범죄 드라마,6.982814
10291,플랜 비,스릴러 드라마 액션 범죄 Made in Europe,5.792202
18485,크라임 스토리: 인도의 형사들,범죄 다큐멘터리,5.840253
2102,로제타,드라마 Made in Europe,7.109471
14394,미성년,드라마,5.840253
16450,프로듀사,코미디 로맨스 드라마 음악,6.132202


In [198]:
중복 = [5217, 11048, 15155]
list(df.iloc[중복]["제목"])

['부당거래', '다만 악에서 구하소서', '인질']

In [199]:
recommendations3(범죄_스릴러, 80)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787]
2 : [8763, 16450, 4677, 6725, 7790, 7279, 4209, 7799, 4744, 4770, 8358, 10419, 5816, 14556, 18147, 4349, 16144, 10549, 5941, 3397, 6492, 6016, 3463, 13193, 6052, 3556]
3 : [5217, 11048, 15155]


,제목,장르,가중치평점
3397,친절한 금자씨,드라마 스릴러 범죄,9.510899
6725,플레이스 비욘드 더 파인즈,드라마 범죄 스릴러,9.277628
11048,다만 악에서 구하소서,액션 범죄 스릴러,9.251573
5217,부당거래,스릴러 범죄 드라마 액션,9.126835
15155,인질,스릴러,8.840253
14556,공작,스릴러 드라마,8.570126
3394,디파티드,스릴러 액션 범죄 드라마,8.489403
16405,살인자 만들기,범죄 다큐멘터리,8.482564
1752,올드보이,스릴러 액션 드라마,8.381451
7799,내부자들,범죄 스릴러 액션 드라마,8.337287


In [200]:

recommendations3(가족_코미디, 20)[["제목", "장르", "가중치평점"]]

[2002, 6699, 1783, 7755]


,제목,장르,가중치평점
70,백설공주와 일곱 난쟁이,판타지 애니메이션 가족 로맨스 스릴러 드라마,7.563903
6903,추억의 마니,드라마 애니메이션 가족,7.518561
1884,이퀼리브리엄,스릴러 액션 드라마 SF,7.380744
613,상하이에서 온 여인,드라마 스릴러 범죄,7.380032
8729,설리: 허드슨강의 기적,드라마,7.375911
592,피노키오,애니메이션 코미디 가족 판타지,7.356063
5269,세상의 모든 계절,드라마 코미디,7.205032
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,7.187778
6700,몬스터 대학교,애니메이션 가족 코미디 판타지 공포,7.184235
4883,몬스터 호텔,판타지 애니메이션 코미디 가족 공포 로맨스,7.079348


In [201]:
df.iloc[[7755]][["제목", "장르"]]

,제목,장르
7755,인사이드 아웃,애니메이션 코미디 드라마 가족 판타지 SF


In [202]:
# ["몬스터 주식회사", "겨울왕국", "슈렉", "인사이드 아웃"]
recommendations3(["인사이드 아웃"], 20)[["제목", "장르", "가중치평점"]]

[7755]


,제목,장르,가중치평점
1884,이퀼리브리엄,스릴러 액션 드라마 SF,7.380744
5269,세상의 모든 계절,드라마 코미디,7.205032
16208,라일리의 세상,코미디 가족 음악 드라마,6.820056
11224,아이 엠 마더,스릴러 드라마 액션 범죄,6.459400
7955,이퀄스,드라마 SF 로맨스,6.068515
17674,인텔리전스,코미디 드라마 역사,5.994430
12426,엔드리스,판타지 코미디 공포 로맨스 스릴러,5.840253
6027,그루피: 사생팬,스릴러 범죄,5.840253
14147,더 가디언: 배틀엔젤,드라마 SF 범죄,5.840253
15025,루프홀,SF 드라마,5.840253


In [161]:
중복 = [8293, 12808, 5135, 11313, 7256, 7299, 9494, 7450, 1942]
list(df.iloc[중복]["제목"])

['고스트 헌터: 얼음 몬스터의 부활',
 '슈퍼 몬스터의 신비한 옛날이야기',
 '백설공주',
 '아이언 마스크: 용패지미',
 '일곱 난쟁이',
 '잠자는 숲속의 미녀와 야수',
 '리틀 프린세스 소피아: 엘레나와 비밀의 아발로 왕국',
 '몬스터 왕국',
 '슈렉 2']

In [210]:
recommendations3(가족_코미디, 20)[["제목", "장르", "가중치평점"]]

[2002, 6699, 1783, 7755]


,제목,장르,가중치평점
70,백설공주와 일곱 난쟁이,판타지 애니메이션 가족 로맨스 스릴러 드라마,7.563903
6903,추억의 마니,드라마 애니메이션 가족,7.518561
1884,이퀼리브리엄,스릴러 액션 드라마 SF,7.380744
613,상하이에서 온 여인,드라마 스릴러 범죄,7.380032
8729,설리: 허드슨강의 기적,드라마,7.375911
592,피노키오,애니메이션 코미디 가족 판타지,7.356063
5269,세상의 모든 계절,드라마 코미디,7.205032
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,7.187778
6700,몬스터 대학교,애니메이션 가족 코미디 판타지 공포,7.184235
4883,몬스터 호텔,판타지 애니메이션 코미디 가족 공포 로맨스,7.079348


In [209]:
recommendations3(가족_코미디, 100)[["제목", "장르", "가중치평점"]]

[2002, 6699, 1783, 7755]
2 : [8293, 12808, 12832, 5049, 5135, 11313, 7256, 7299, 9494, 7450, 1942]


,제목,장르,가중치평점
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,9.187778
1873,파이트 클럽,드라마,8.788208
6662,인터스텔라,SF 드라마 스릴러,8.589005
5049,슈렉 포에버,판타지 애니메이션 코미디 가족 로맨스,8.290759
3314,업,코미디 드라마 가족 애니메이션,8.190475
17670,사이코지만 괜찮아,드라마 코미디 로맨스,8.155286
1702,화양연화,드라마 로맨스,8.035436
493,우리 생애 최고의 해,드라마 로맨스 전쟁,7.960939
17343,드래곤 프린스,액션 SF 애니메이션 드라마 가족 판타지 스릴러,7.954827
208,로마의 휴일,로맨스 코미디,7.936942


In [205]:
범죄_스릴러_가족_코미디

['타짜', '신세계', '검사외전', '베테랑', '몬스터 주식회사', '겨울왕국', '슈렉', '인사이드 아웃']

In [206]:

recommendations3(범죄_스릴러_가족_코미디, 20)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787, 2002, 6699, 1783, 7755]
2 : [10549, 5941]


,제목,장르,가중치평점
15542,프리즌 브레이크,액션 드라마 스릴러 범죄 Made in Europe,8.280361
10549,꾼,액션 범죄 스릴러,7.992202
5941,해결사,스릴러 액션 범죄 드라마,7.840253
7771,시카리오: 암살자의 도시,스릴러 드라마 액션 범죄,7.582705
70,백설공주와 일곱 난쟁이,판타지 애니메이션 가족 로맨스 스릴러 드라마,7.563903
6903,추억의 마니,드라마 애니메이션 가족,7.518561
3397,친절한 금자씨,드라마 스릴러 범죄,7.510899
406,폴리스 스토리,스릴러 액션 코미디 범죄,7.409757
1884,이퀼리브리엄,스릴러 액션 드라마 SF,7.380744
613,상하이에서 온 여인,드라마 스릴러 범죄,7.380032


In [166]:
중복 = [16450, 8293, 4209, 8358, 10419, 14556, 4349, 10549, 6492, 12808, 8763, 4677, 6725, 4744, 4770, 13193, 2967, 5135, 11313, 7256, 7279, 7299, 3255, 9494, 7450, 3397, 3463, 3556, 7790, 7799, 5816, 18147, 16144, 5941, 6016, 1942, 6052]
list(df.iloc[중복]["제목"])

['프로듀사',
 '고스트 헌터: 얼음 몬스터의 부활',
 '정승필 실종사건',
 '탐정: 더 비기닝',
 '반드시 잡는다',
 '공작',
 '유감스러운 도시',
 '꾼',
 '표적',
 '슈퍼 몬스터의 신비한 옛날이야기',
 '아수라',
 '우리 동네',
 '플레이스 비욘드 더 파인즈',
 '그림자 살인',
 '무방비 도시',
 '암수살인',
 '산사초',
 '백설공주',
 '아이언 마스크: 용패지미',
 '일곱 난쟁이',
 '사이코메트리',
 '잠자는 숲속의 미녀와 야수',
 '주노명 베이커리',
 '리틀 프린세스 소피아: 엘레나와 비밀의 아발로 왕국',
 '몬스터 왕국',
 '친절한 금자씨',
 '극락도 살인사건',
 '블랙 달리아',
 '오피스',
 '내부자들',
 '특수본',
 '날아라 개천용',
 '내 연애의 모든 것',
 '해결사',
 '프론트',
 '슈렉 2',
 '시체가 돌아왔다']

In [207]:

recommendations3(범죄_스릴러_가족_코미디, 80)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787, 2002, 6699, 1783, 7755]
2 : [16450, 8293, 4209, 8358, 10419, 14556, 4349, 10549, 6492, 12808, 8763, 4677, 6725, 4744, 4770, 13193, 2967, 5135, 11313, 7256, 7279, 7299, 3255, 9494, 7450, 3397, 3463, 3556, 7790, 7799, 5816, 18147, 16144, 5941, 6016, 1942, 6052]
3 : [11048, 15155, 5217]


,제목,장르,가중치평점
3397,친절한 금자씨,드라마 스릴러 범죄,9.510899
6725,플레이스 비욘드 더 파인즈,드라마 범죄 스릴러,9.277628
11048,다만 악에서 구하소서,액션 범죄 스릴러,9.251573
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,9.187778
5217,부당거래,스릴러 범죄 드라마 액션,9.126835
15155,인질,스릴러,8.840253
14556,공작,스릴러 드라마,8.570126
3394,디파티드,스릴러 액션 범죄 드라마,8.489403
16405,살인자 만들기,범죄 다큐멘터리,8.482564
1752,올드보이,스릴러 액션 드라마,8.381451


In [168]:
중복 = [4118, 4770, 11048, 15155, 5217]
list(df.iloc[중복]["제목"])

['혈의 누', '무방비 도시', '다만 악에서 구하소서', '인질', '부당거래']

In [167]:

recommendations3(범죄_스릴러_가족_코미디, 100)[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787, 2002, 6699, 1783, 7755]
2 : [16450, 8293, 4209, 8358, 10419, 14556, 4349, 10549, 6492, 8648, 12808, 12832, 8763, 4677, 6725, 4744, 10956, 13193, 5005, 2967, 5049, 7116, 5135, 11313, 7256, 7279, 7299, 3206, 3255, 9494, 7450, 3397, 13693, 3463, 3556, 5738, 7790, 7799, 9888, 5816, 7872, 18147, 16144, 5941, 6016, 1942, 6052, 16301]
3 : [4118, 4770, 11048, 15155, 5217]


,제목,장르,가중치평점
3397,친절한 금자씨,드라마 스릴러 범죄,9.510899
6725,플레이스 비욘드 더 파인즈,드라마 범죄 스릴러,9.277628
11048,다만 악에서 구하소서,액션 범죄 스릴러,9.251573
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,9.187778
5217,부당거래,스릴러 범죄 드라마 액션,9.126835
5005,황해,스릴러 액션 범죄 드라마,9.056709
4118,혈의 누,스릴러,8.992202
4770,무방비 도시,스릴러 액션 범죄,8.840253
15155,인질,스릴러,8.840253
1873,파이트 클럽,드라마,8.788208


In [135]:
import pandas as pd
import numpy as np

def recommendations(titles, genre = "특징"):
    # titles 예능인지 아닌지
    
    indexes = []
    for title in titles:
        indexes.append(df[df.제목 == title].index[0])
    similar_indexes = pd.read_csv(f"./data/{genre}_matrix_21.csv")
    similar_indexes = np.array(similar_indexes)[indexes,:]
    similar_indexes_ls = list(similar_indexes.reshape(-1))
    similar_indexes = set(similar_indexes_ls)  # 중복된 영화들을 제거/ 가중치 부여 # ["마블", "마블", "대부", "배트맨"]
    similar_indexes = np.array(list(similar_indexes))
    for i in indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거

    # return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20]
    return df.iloc[similar_indexes][:20]
    
def recommendations2(titles, genre = "특징"):
    # titles 예능인지 아닌지
    
    indexes = []
    for title in titles:
        indexes.append(df[df.제목 == title].index[0])
    print(indexes)
    similar_indexes = pd.read_csv(f"./data/{genre}_matrix_105.csv")
    similar_indexes = np.array(similar_indexes).reshape((-1, 105))
    similar_indexes = similar_indexes[indexes,:]
    similar_indexes_ls = list(similar_indexes.reshape(-1))
    similar_indexes = set(similar_indexes_ls)  # 중복된 영화들을 제거/ 가중치 부여 # ["마블", "마블", "대부", "배트맨"]
    
    similar_indexes = np.array(list(similar_indexes))
    for i in indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거

    counter = {}
    for index in similar_indexes:
        counter[index] = similar_indexes_ls.count(index)
    
    ls = [[] for i in range(max(counter.values()))]
    for k, v in counter.items():
        ls[v - 1] += [k]
    
    for i, l in enumerate(ls):
        if i > 0:
            print(f"{i+1} : {l}")
        

    result = df.iloc[similar_indexes]
    for k, v in counter.items():
        if v > 1:
            result["가중치평점"][k] += v
    
    return result.sort_values("가중치평점", ascending=False)[:20]
    # return df.iloc[similar_indexes][:20]

In [85]:
p = "라따뚜이"
df[df.제목.apply(lambda x: p in x)]["제목"]

3298    라따뚜이
Name: 제목, dtype: object

In [120]:
titles = ["몬스터 주식회사", "겨울왕국", "슈렉", "나 홀로 집에", "인사이드 아웃",
          "알라딘", "라따뚜이"]

In [129]:
indexes = [4099, 6168, 4188, 8293, 12515, 4344, 12639, 12808, 12832, 6717, 2863, 11090, 13150, 9088, 904, 9144, 5049, 17359, 5135, 11313, 7256, 7299, 7300, 9494, 7450, 5633, 5953, 1942]
list(df.iloc[indexes].sort_values("가중치평점", ascending=False)["제목"])

['도로헤도로',
 '슈렉 2',
 '바그다드의 도둑',
 '녹차의 맛',
 '알라딘',
 '인생을 애니메이션처럼',
 '슈렉 포에버',
 '사일런트 하트',
 '밤의 이야기',
 '숲속으로',
 '고스트 헌터: 얼음 몬스터의 부활',
 '로덴시아: 마법왕국의 전설',
 '몬스터 왕국',
 '리틀 프린세스 소피아: 엘레나와 비밀의 아발로 왕국',
 '신데렐라 3',
 '아기곰 보보 구출 대작전',
 '잃어버린 제국을 찾아서',
 '백 투 더 씨',
 '드래곤 프린세스',
 '몬스터 파크',
 '슈퍼 몬스터의 신비한 옛날이야기',
 '알라딘 2',
 '페어러브',
 '백설공주',
 '일곱 난쟁이',
 '두갈: 마법의 회전목마',
 '잠자는 숲속의 미녀와 야수',
 '아이언 마스크: 용패지미']

In [130]:
recommendations2(titles)[["제목", "장르", "가중치평점"]]

[2002, 6699, 1783, 0, 7755, 309, 3298]
2 : [4099, 6168, 4188, 8293, 12515, 4344, 12639, 12808, 12832, 6717, 2863, 11090, 13150, 9088, 904, 9144, 5049, 17359, 5135, 11313, 7256, 7299, 7300, 9494, 7450, 5633, 5953, 1942]


,제목,장르,가중치평점
17359,도로헤도로,액션 SF 판타지 공포 애니메이션 코미디,9.278274
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,9.187778
904,바그다드의 도둑,판타지 가족 로맨스,9.085063
2863,녹차의 맛,드라마 코미디 판타지,8.956101
11090,알라딘,판타지 액션 로맨스 코미디 가족 드라마,8.883245
1873,파이트 클럽,드라마,8.788208
9088,인생을 애니메이션처럼,다큐멘터리 가족 판타지 로맨스 코미디 드라마 애니메이션 공포,8.762335
6662,인터스텔라,SF 드라마 스릴러,8.589005
16563,디스 이즈 어스,드라마 코미디 로맨스,8.511688
15789,슈츠,드라마 코미디,8.373679


In [136]:
recommendations(["겨울왕국"])[["제목", "장르", "가중치평점"]]

,제목,장르,가중치평점
7690,드래곤 기사단,애니메이션 Made in Europe,5.840253
2454,안나 카레니나,드라마 로맨스,6.095668
8087,겨울왕국 열기,코미디,5.840253
1048,패러딘 부인의 사랑,스릴러 드라마 범죄 로맨스,6.311501
14874,보더라인,드라마 로맨스 스릴러,5.840253
18473,숨겨진 세상: 겨울왕국2 메이킹,다큐멘터리 가족,5.840253
7978,디센던츠,판타지 코미디 가족,6.298228
4282,더 리턴,스릴러 드라마 공포,5.840253
11074,안나,스릴러 액션 Made in Europe,6.559480
709,데미지,로맨스 드라마,6.617191


In [131]:
recommendations(titles)[["제목", "장르", "가중치평점"]]

,제목,장르,가중치평점
16419,셰프의 테이블,다큐멘터리,7.908945
6,구니스,액션 코미디 가족,7.770194
70,백설공주와 일곱 난쟁이,판타지 애니메이션 가족 로맨스 스릴러 드라마,7.563903
6903,추억의 마니,드라마 애니메이션 가족,7.518561
1830,데블스 에드버킷,드라마 스릴러 공포,7.481246
4923,케빈에 대하여,스릴러 드라마,7.455443
1884,이퀼리브리엄,스릴러 액션 드라마 SF,7.380744
613,상하이에서 온 여인,드라마 스릴러 범죄,7.380032
8729,설리: 허드슨강의 기적,드라마,7.375911
592,피노키오,애니메이션 코미디 가족 판타지,7.356063


In [92]:
df.iloc[[6168, 4188, 12639, 12808, 13150, 904, 5135, 7256, 7300, 9494, 5633, 1942]][["제목", "장르"]]

,제목,장르
6168,백 투 더 씨,애니메이션 코미디 가족
4188,신데렐라 3,판타지
12639,알라딘 2,코미디 판타지 Made in Europe
12808,슈퍼 몬스터의 신비한 옛날이야기,애니메이션 가족
13150,드래곤 프린세스,판타지 가족 애니메이션
904,바그다드의 도둑,판타지 가족 로맨스
5135,백설공주,가족 판타지 액션 코미디 드라마 다큐멘터리 Made in Europe SF 스릴러
7256,일곱 난쟁이,판타지 Made in Europe 애니메이션 코미디 가족
7300,사일런트 하트,드라마 Made in Europe
9494,리틀 프린세스 소피아: 엘레나와 비밀의 아발로 왕국,판타지


In [90]:
recommendations2(titles)[["제목", "장르", "가중치평점"]]

[2002, 6699, 1783, 0, 7755, 309, 3298]
1 : [8194, 4099, 2053, 6, 8203, 16395, 4109, 4110, 13, 18443, 8212, 2072, 8219, 29, 35, 16419, 18473, 8234, 12331, 6191, 2104, 70, 14406, 4169, 10323, 18516, 12380, 6241, 8293, 14451, 4213, 16502, 8311, 10361, 12426, 16526, 16540, 6305, 2223, 16563, 184, 12473, 4282, 4288, 6342, 205, 208, 4304, 8405, 4315, 2272, 12515, 12516, 10474, 243, 12532, 4340, 10504, 16653, 4392, 2354, 8505, 14653, 8511, 10568, 341, 10583, 6490, 2399, 16739, 361, 18794, 12664, 385, 12681, 2454, 8598, 12698, 416, 6560, 8612, 14757, 16811, 6576, 6585, 2492, 12739, 10698, 10704, 12753, 8661, 2529, 2531, 8676, 6629, 18924, 12783, 10742, 6650, 12798, 10752, 16901, 14855, 12809, 2572, 8729, 14874, 12832, 8737, 6700, 12850, 18997, 16951, 12856, 6717, 12863, 6728, 592, 2642, 12888, 12891, 19040, 19042, 19044, 613, 2662, 616, 19056, 6770, 8821, 2684, 2686, 4735, 2688, 8830, 17027, 19085, 14991, 19087, 12945, 663, 12957, 17068, 15025, 2741, 12986, 700, 6845, 6848, 709, 17096, 6862, 1

,제목,장르,가중치평점
16563,디스 이즈 어스,드라마 코미디 로맨스,9.511688
15789,슈츠,드라마 코미디,9.373679
1942,슈렉 2,애니메이션 코미디 가족 판타지 로맨스,9.187778
17670,사이코지만 괜찮아,드라마 코미디 로맨스,9.155286
904,바그다드의 도둑,판타지 가족 로맨스,9.085063
7759,룸,드라마 스릴러 Made in Europe,9.077346
2046,인크레더블,애니메이션 액션 가족,8.987516
35,JFK,스릴러 드라마 역사,8.943536
208,로마의 휴일,로맨스 코미디,8.936942
16419,셰프의 테이블,다큐멘터리,8.908945


In [115]:
recommendations(["타짜", "신세계", "검사외전", "베테랑"])[["제목", "장르", "가중치평점"]]

,제목,장르,가중치평점
15542,프리즌 브레이크,액션 드라마 스릴러 범죄 Made in Europe,8.280361
7771,시카리오: 암살자의 도시,스릴러 드라마 액션 범죄,7.582705
3397,친절한 금자씨,드라마 스릴러 범죄,7.510899
406,폴리스 스토리,스릴러 액션 코미디 범죄,7.409757
3396,모범시민,스릴러 드라마 액션 범죄,7.378412
16852,당신이 잠든 사이에,SF 로맨스 스릴러 코미디 드라마 판타지,7.262335
6315,필스,범죄 액션 코미디 드라마 Made in Europe,7.053771
8834,인필트레이터: 잠입자들,스릴러 범죄 드라마,6.938147
796,폴리스 스토리 2: 구룡의 눈,스릴러 액션 코미디 범죄,6.870955
7753,뷰티 인사이드,판타지 드라마 로맨스,6.776101


In [94]:
df.iloc[[8763, 5217, 7790, 4209, 7799, 4744, 4770, 10419, 5816, 14556, 18147, 4349, 15155, 10549, 5941, 3397, 6492, 6016, 3463, 13193, 11048]][["제목", "장르"]]

,제목,장르
8763,아수라,범죄 드라마 스릴러 액션
5217,부당거래,스릴러 범죄 드라마 액션
7790,오피스,스릴러 공포
4209,정승필 실종사건,코미디
7799,내부자들,범죄 스릴러 액션 드라마
4744,그림자 살인,스릴러
4770,무방비 도시,스릴러 액션 범죄
10419,반드시 잡는다,스릴러 범죄 드라마
5816,특수본,범죄 스릴러
14556,공작,스릴러 드라마


In [93]:
recommendations2(["타짜", "신세계", "검사외전", "베테랑"])[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787]
1 : [2048, 13314, 3, 11266, 9733, 5128, 11275, 16405, 4118, 11800, 17432, 4632, 543, 15908, 16936, 5165, 4659, 10291, 14902, 5689, 11324, 4669, 17983, 16450, 4677, 6725, 18501, 3656, 7753, 6729, 11339, 7241, 9294, 10835, 11866, 7771, 7773, 14434, 5732, 4199, 9321, 9839, 2162, 4723, 7795, 7279, 4215, 2688, 8834, 8837, 3206, 3721, 17035, 4235, 3725, 5262, 14994, 661, 17046, 5787, 10908, 7838, 9888, 6305, 3747, 17059, 8358, 3239, 6310, 6315, 2221, 16046, 6831, 15536, 16561, 1715, 15542, 3255, 6327, 3770, 8389, 17605, 4296, 6345, 17101, 11983, 2768, 5841, 11478, 18136, 7896, 8924, 7902, 14047, 6883, 5348, 12004, 5863, 12007, 12010, 3819, 16618, 9966, 16622, 3312, 242, 18676, 3317, 18166, 762, 5884, 9476, 7945, 9482, 6411, 6924, 16144, 11537, 2328, 10010, 2331, 796, 17692, 8990, 16159, 11553, 13612, 6447, 1329, 9526, 4919, 9018, 3394, 3396, 9030, 8006, 14153, 10059, 8012, 8020, 6485, 16222, 16742, 7016, 11625, 16234, 4974, 17262, 15216, 5490, 10100, 15733, 13180, 49

,제목,장르,가중치평점
3397,친절한 금자씨,드라마 스릴러 범죄,9.510899
3394,디파티드,스릴러 액션 범죄 드라마,9.489403
16405,살인자 만들기,범죄 다큐멘터리,9.482564
1715,스내치,범죄 코미디,9.288059
15542,프리즌 브레이크,액션 드라마 스릴러 범죄 Made in Europe,9.280361
11048,다만 악에서 구하소서,액션 범죄 스릴러,9.251573
3511,엘 시크레토: 비밀의 눈동자,드라마 Made in Europe 로맨스 스릴러,9.154180
15536,화이트 칼라,범죄 코미디 드라마,9.126830
2328,무간도,스릴러 드라마 액션 범죄,8.931437
450,허슬러,드라마 스포츠,8.895916


In [99]:
df.iloc[[16450, 4209, 8358, 10419, 14556, 4349, 10549, 6492, 8648, 8763, 4677, 6725, 4744, 13193, 5005, 7116, 7279, 3206, 3397, 13693, 3463, 3556, 5738, 7790, 7799, 9888, 5816, 7872, 18147, 16144, 5941, 6016, 6052, 16301]][["제목", "장르"]]

,제목,장르
16450,프로듀사,코미디 로맨스 드라마 음악
4209,정승필 실종사건,코미디
8358,탐정: 더 비기닝,스릴러 코미디 범죄
10419,반드시 잡는다,스릴러 범죄 드라마
14556,공작,스릴러 드라마
4349,유감스러운 도시,코미디 액션
10549,꾼,액션 범죄 스릴러
6492,표적,스릴러 액션
8648,살인의뢰,드라마 스릴러 범죄
8763,아수라,범죄 드라마 스릴러 액션


In [100]:
df.iloc[[4118, 4770, 11048, 15155, 5217]][["제목", "장르"]]

,제목,장르
4118,혈의 누,스릴러
4770,무방비 도시,스릴러 액션 범죄
11048,다만 악에서 구하소서,액션 범죄 스릴러
15155,인질,스릴러
5217,부당거래,스릴러 범죄 드라마 액션


In [98]:
recommendations2(["타짜", "신세계", "검사외전", "베테랑"])[["제목", "장르", "가중치평점"]]

[3938, 6255, 9001, 7787]
1 : [2048, 3, 16405, 2081, 10291, 18485, 2102, 14394, 18501, 6222, 6223, 18510, 4184, 14434, 4199, 4208, 2162, 4215, 4229, 4235, 4249, 6305, 6310, 18602, 6315, 2221, 16561, 6324, 6327, 18626, 8389, 6343, 4296, 6345, 4305, 6357, 4311, 16618, 16622, 242, 18676, 6397, 14591, 6401, 6411, 4366, 16664, 2328, 8473, 2331, 4398, 6447, 6485, 18778, 16742, 8551, 6502, 18803, 4485, 406, 6554, 6568, 6579, 6581, 6582, 4539, 16830, 16833, 450, 6593, 16838, 4556, 6609, 16852, 16859, 476, 4632, 543, 4643, 16936, 2607, 10802, 4659, 14902, 10808, 4669, 6729, 10835, 4723, 2688, 8834, 8837, 4742, 17035, 14994, 661, 17046, 10908, 12963, 17059, 6824, 6831, 2739, 8890, 17085, 10956, 17101, 2768, 17111, 8924, 6883, 762, 11003, 6924, 784, 2836, 4891, 796, 8990, 15141, 4919, 9018, 17221, 9030, 17235, 11095, 11096, 2915, 7016, 7020, 4974, 17262, 15216, 15221, 7033, 13180, 2942, 4992, 897, 15255, 9112, 2967, 5021, 9129, 17322, 7090, 15290, 11194, 5066, 17376, 11233, 1003, 11244, 17391, 173

,제목,장르,가중치평점
3397,친절한 금자씨,드라마 스릴러 범죄,9.510899
3394,디파티드,스릴러 액션 범죄 드라마,9.489403
16405,살인자 만들기,범죄 다큐멘터리,9.482564
1752,올드보이,스릴러 액션 드라마,9.381451
1715,스내치,범죄 코미디,9.288059
15542,프리즌 브레이크,액션 드라마 스릴러 범죄 Made in Europe,9.280361
6725,플레이스 비욘드 더 파인즈,드라마 범죄 스릴러,9.277628
11048,다만 악에서 구하소서,액션 범죄 스릴러,9.251573
17235,킹덤,액션 SF 스릴러 드라마 역사 공포,9.167296
3511,엘 시크레토: 비밀의 눈동자,드라마 Made in Europe 로맨스 스릴러,9.154180
